In [18]:
import pandas as pd
import numpy as np
import requests
import json

In [14]:
addresses = pd.read_csv('addresses.csv')['full_address'].tolist()

In [28]:
def geocode(address: str):
    # The address of your Photon server, accessed through the SSH tunnel on your MacBook
    photon_api_url_on_macbook = "http://localhost:8000/api"
    
    # Construct the query parameters
    params_forward = {
        'q': address,
        'limit': 1 # Optionally limit the number of results
    }
    
    try:
        response_forward = requests.get(photon_api_url_on_macbook, params=params_forward)
        response_forward.raise_for_status()  # Raise an exception for HTTP errors (4xx or 5xx)
        
        geocoding_results = response_forward.json() # Parse the JSON response
        
        #print(f"Geocoding results for {address}:")
        
        # can extract coordinates or other info from the results
        if geocoding_results.get('features'):
            first_result = geocoding_results['features'][0]
            coordinates = first_result.get('geometry', {}).get('coordinates')
            properties = first_result.get('properties', {})
            
            #print(f"Name: {properties.get('name')}, City: {properties.get('city')}, Country: {properties.get('country')}")
            
        #return json.dumps(geocoding_results, indent=2)
        return coordinates
            
    except requests.exceptions.RequestException as e:
        print(f"Error connecting to Photon server: {e}")
    except json.JSONDecodeError:
        print(f"Error decoding JSON response: {response_forward.text}")

In [42]:
addresses[215]
problem_address = '11816-11998 CORLISS AVE NE SEATTLE, WA'

In [23]:
coords_array = np.zeros((703,2))

In [47]:
%%time
for i, address in enumerate(addresses):
    try:
        coords_array[i] = geocode(address)
    except UnboundLocalError:
        print(i)

215
CPU times: user 2.23 s, sys: 1.04 s, total: 3.27 s
Wall time: 1min 53s


In [30]:
coords_array

array([[-122.35584   ,   47.7220511 ],
       [-122.35584   ,   47.7220511 ],
       [-122.3559837 ,   47.7216444 ],
       ...,
       [-122.2951472 ,   47.7093109 ],
       [-122.29516345,   47.7090713 ],
       [-122.29515651,   47.70890285]], shape=(703, 2))

In [36]:
df = pd.DataFrame({
    'address': addresses,
    'latitude': coords_array[:,0],
    'longitude': coords_array[:,1]
})

In [40]:
df.drop(215)

,address,latitude,longitude
0,"12717 B GREENWOOD AVE N SEATTLE, WA",-122.355840,47.722051
1,"12717 A GREENWOOD AVE N SEATTLE, WA",-122.355840,47.722051
2,"12701 GREENWOOD AVE N SEATTLE, WA",-122.355984,47.721644
3,"12704 PALATINE AVE N SEATTLE, WA",-122.356552,47.721630
4,"12507 A GREENWOOD AVE N SEATTLE, WA",-122.355991,47.721153
...,...,...,...
698,"11039 31 AVE NE SEATTLE, WA",-122.295245,47.709597
699,"11035 31 AVE NE SEATTLE, WA",-122.295244,47.709486
700,"11027 31 AVE NE SEATTLE, WA",-122.295147,47.709311
701,"11021 31 AVE NE SEATTLE, WA",-122.295163,47.709071
